In [2]:
import pandas as pd

In [3]:
df = pd.read_parquet('m5_clean_daily.parquet')
df[:5]

,id,item_id,dept_id,cat_id,store_id,state_id,d,demand,date,wm_yr_wk,...,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,day_of_week,week_of_year,is_weekend,sell_price
7,HOBBIES_1_008_CA_1_validation,HOBBIES_1_008,HOBBIES_1,HOBBIES,CA_1,CA,d_1,12,2011-01-29,11101,...,None,None,None,0,0,0,5,4,1,0.46
8,HOBBIES_1_009_CA_1_validation,HOBBIES_1_009,HOBBIES_1,HOBBIES,CA_1,CA,d_1,2,2011-01-29,11101,...,None,None,None,0,0,0,5,4,1,1.56
9,HOBBIES_1_010_CA_1_validation,HOBBIES_1_010,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,...,None,None,None,0,0,0,5,4,1,3.17
11,HOBBIES_1_012_CA_1_validation,HOBBIES_1_012,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,...,None,None,None,0,0,0,5,4,1,5.98
14,HOBBIES_1_015_CA_1_validation,HOBBIES_1_015,HOBBIES_1,HOBBIES,CA_1,CA,d_1,4,2011-01-29,11101,...,None,None,None,0,0,0,5,4,1,0.70


In [4]:
df.shape

(46027957, 25)

In [8]:
lags = [1, 7, 14, 28]

for lag in lags:
    df[f"lag_{lag}"] = (
        df
        .groupby(["item_id", "store_id"])["demand"]
        .shift(lag)
    )


In [9]:
df["rolling_mean_7"] = (
    df
    .groupby(["item_id", "store_id"])["demand"]
    .shift(1)
    .rolling(7)
    .mean()
)


In [10]:
df.to_parquet("m5_clean_daily2.parquet")

In [ ]:
df = pd.read_parquet('m5_clean_daily2.parquet')

In [14]:
train = df[df["date"] < "2016-01-01"]
valid = df[df["date"] >= "2016-01-01"]


In [15]:
y_train = train ['demand']
y_val = valid['demand']

x_train = train.drop(columns=['demand'])
x_val = valid.drop(columns=['demand'])

In [16]:
from sklearn.ensemble import GradientBoostingRegressor

model = GradientBoostingRegressor().fit(x_train,y_train)

MemoryError: Unable to allocate 9.19 GiB for an array with shape (29, 42522120) and data type object